In [2]:
import re
import pickle
import sentencepiece as spm
from tqdm import tqdm

In [4]:
sp = spm.SentencePieceProcessor()
sp.load('Tokenizer/all_reviews/global_spm.model')

True

In [23]:
with open('all_reviews.txt', 'rt') as f:
    all_reviews = f.read().splitlines()

In [24]:
def clean_reivew(review):
    review = ''.join(re.compile('[가-힣.!? ]').findall(review))
    
    review = re.sub(r'\!+', '!', review)
    review = re.sub(r'\?+', '?', review)
    review = re.sub(r'\.+', '.', review)
    review = re.sub(r'\([^)]*\)', '', review)    # 괄호 안 내용 삭제
    review = re.sub(r'\[[^)]*\]', '', review)    # 대괄호 안 내용 삭제
    
    return review

In [ ]:
prep_reviews = [[re.sub('▁','',token) for token in sp.EncodeAsPieces(clean_reivew(review))] for review in tqdm(all_reviews) if review]

In [ ]:
with open('prep_all_reviews', 'wb') as f:
    pickle.dump(prep_reviews, f)

In [ ]:
with open('prep_all_reviews', 'rb') as f:
    prep_reviews = pickle.load(f)

In [ ]:
from gensim.models import FastText

In [10]:
prep_reviews[:100]

[['포장', '상태도', '좋고', '맛있어요'],
 ['포장상태', '고', '좋고', '맛있어요'],
 ['딱',
  '예상했던',
  '맛이',
  '지만',
  '쎄',
  '븐',
  '이나',
  '이런',
  '데',
  '보다',
  '훨씬',
  '싸고',
  '냉장',
  '상품',
  '이라',
  '유통기한은',
  '짧',
  '지만',
  '만들어진',
  '날짜',
  '가',
  '최근',
  '이라',
  '정말',
  '좋아요'],
 [],
 ['두번째구매', '에요', '맛있고', '먹을만해요'],
 ['배송도빠르고', '만족합니다'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['배송', '빨랐고', '맛있어요'],
 ['배송은', '빠릅니다'],
 ['참', '맛있는', '햄버거', '를', '먹게', '되서', '행복', '합니다'],
 ['참', '맛난', '햄버거', '를', '먹어서', '행복', '해요'],
 ['오랜만에',
  '주문해서',
  '먹었습니다',
  '종류가',
  '많이',
  '늘',
  '었네요',
  '맛이',
  '궁금해서',
  '종류별로',
  '전부',
  '주문했는데',
  '여',
  '잔',
  '히',
  '맛있어요',
  '아쉬운점은',
  '야채',
  '가',
  '조금',
  '더',
  '들어갔',
  '으면',
  '합니다'],
 ['리뷰', '쓰는', '거', '너무', '싫', '음', '맛있으면', '끝난', '거', '아님'],
 ['실망', '시키지', '않네요', '여전히', '맛있어요'],
 ['두고두고', '먹으려', '많이', '주문했는데', '맛있', '으니', '자꾸', '손이', '갑니다'],
 ['계란', '양', '배', '추', '준비', '하시고', '같이', '넣어서', '드', '시면', '더욱', '더', '맛있어요'],
 ['맛', '포장', '배송', '가격', '갑입니다', '재구매의사', '번창', '

In [3]:
fasttext = FastText(sg=1, size=384, workers=8, word_ngrams=2, min_count=200, window=5, alpha = 0.05, max_n=4)

NameError: name 'FastText' is not defined

In [ ]:
fasttext.build_vocab(sentences=prep_reviews)

In [ ]:
fasttext.wv.index2word

In [ ]:
fasttext.train(sentences=prep_reviews, total_examples=fasttext.corpus_count, epochs=5)

In [14]:
fasttext.save('0806_fasttext_model_all_reviews_bigram_skipgram_epoch=5_WS=5_alpha=0.05_mincount=150_maxn=4')

In [13]:
fasttext.wv.similar_by_word("트러블", topn=30)

[('피부트러블', 0.8399746417999268),
 ('트러블이', 0.7926641702651978),
 ('트러블도', 0.7712335586547852),
 ('뾰루지', 0.7619308233261108),
 ('여드름', 0.7130717039108276),
 ('좁쌀', 0.6828883290290833),
 ('뾰루지가', 0.6570552587509155),
 ('민감성', 0.6281565427780151),
 ('드름', 0.6062622666358948),
 ('피부인데', 0.5967861413955688),
 ('피부자극', 0.5822704434394836),
 ('피부염', 0.5617694854736328),
 ('각질', 0.5585975050926208),
 ('부작용', 0.5570420026779175),
 ('홍조', 0.547311544418335),
 ('피부진정', 0.5452874898910522),
 ('속당김', 0.539918065071106),
 ('진정효과', 0.5374125838279724),
 ('자극', 0.535317063331604),
 ('피부', 0.5340607166290283),
 ('알러지', 0.5324243307113647),
 ('염증', 0.5305849313735962),
 ('가려움', 0.5274947881698608),
 ('피부라', 0.5253651142120361),
 ('당김', 0.5195484161376953),
 ('자극없고', 0.5193876028060913),
 ('흉터', 0.5149660110473633),
 ('진정', 0.5148909687995911),
 ('건조함', 0.5143353939056396),
 ('순해서', 0.5135738253593445)]

In [6]:
model = FastText.load('fasttext_model/fasttext_model_all_reviews_bigram_skipgram_epoch=5')

In [29]:
model.wv.most_similar("싫어하는데", topn=30)

[('안좋아하는데', 0.8222872018814087),
 ('싫어하는', 0.708359956741333),
 ('좋아하는데', 0.6959294676780701),
 ('싫어해서', 0.6942651867866516),
 ('좋아하지', 0.6088773012161255),
 ('싫어', 0.5896216034889221),
 ('싫은', 0.5624194145202637),
 ('좋아하지만', 0.5536218881607056),
 ('원했는데', 0.5153918266296387),
 ('찾고있었는데', 0.4972422420978546),
 ('선호하는', 0.48062437772750854),
 ('느낌인데', 0.46815192699432373),
 ('찾았는데', 0.4614163637161255),
 ('않는데', 0.457257479429245),
 ('끈적이는', 0.44762784242630005),
 ('선호', 0.4461081922054291),
 ('싫', 0.4460225999355316),
 ('싫어서', 0.4450443983078003),
 ('좋아하시는', 0.43521058559417725),
 ('싫다', 0.4306098222732544),
 ('좋아하는', 0.4247414767742157),
 ('힘든데', 0.42146819829940796),
 ('안좋아', 0.4204910099506378),
 ('못하는데', 0.4200853109359741),
 ('편인데', 0.4150669574737549),
 ('좋아하고', 0.4076610207557678),
 ('안쓰는데', 0.40645095705986023),
 ('좋아해서', 0.4058074355125427),
 ('하거든요', 0.40001681447029114),
 ('불편했는데', 0.3949490189552307)]

In [5]:
ft = FastText.load('0804_fasttext_model_all_reviews_trigram_skipgram_epoch=10_WS=2')

In [6]:
ft.wv.syn0.shape

/home/literai-deploy/anaconda3/envs/gale/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  """Entry point for launching an IPython kernel.


(26089, 384)

In [12]:
import os

In [13]:
os.stat('0805_fasttext_model_all_reviews_bigram_skipgram_epoch=5_WS=5_alpha=0.05')

os.stat_result(st_mode=33204, st_ino=18219021, st_dev=66306, st_nlink=1, st_uid=1000, st_gid=1000, st_size=232253428, st_atime=1596612700, st_mtime=1596612609, st_ctime=1596612609)